# Create a new private Collection

The script in this notebook creates a new private Collection.

In order to use this script, you must have a Curation API key (obtained from upper-righthand dropdown in the CZ CELLxGENE Discover data portal after logging in).

### Import dependencies

In [ ]:
library("readr")
library("httr")
library("stringr")
library("rjson")

#### <font color='#bc00b0'>Please fill in the required values:</font>

<font color='#bc00b0'>(Required) Provide the path to your api key file</font>

In [ ]:
api_key_file_path <- "path/to/api-key.txt"

### Specify domain (and API url)

In [ ]:
domain_name <- "cellxgene.cziscience.com"
site_url <- str_interp("https://${domain_name}")
api_url_base <- str_interp("https://api.${domain_name}")

### Use API key to obtain a temporary access token

In [ ]:
api_key <- read_file(api_key_file_path)
access_token_path <- "/curation/v1/auth/token"
access_token_url <- str_interp("${api_url_base}${access_token_path}")
res <- POST(url=access_token_url, add_headers(`x-api-key`=api_key))
stop_for_status(res)
access_token <- content(res)$access_token

##### (optional, debug) verify status code of response

In [ ]:
print(res$status_code)

<font color='#bc00b0'>(Required) Provide the updated Collection form metadata for your new Collection. If included, the DOI must be in the form of a [CURIE Reference](https://www.w3.org/TR/2010/NOTE-curie-20101216/#s_syntax), and if it cannot be found on Crossref or is invalid, the entire request will be rejected.</font>

In [ ]:
collection_form_metadata <- list(
  name = "Name",  # Required
  description = "A sample description",  # Required
  contact_name = "First Last",  # Required
  contact_email = "firstlast@email.com",  # Required
  doi = "10.1093/ajae/aaq063",
  links = list(
      list(
          link_name = "sample protocol link",
          link_url = "https://www.sample.link",
          link_type = "PROTOCOL"
      ),
      list(
          link_name = "sample lab website",
          link_url = "https://www.lab.website",
          link_type = "LAB_WEBSITE"
      )
  )
)

### Formulate request and create new private Collection

In [ ]:
collections_path <- "/curation/v1/collections"
url <- str_interp("${api_url_base}${collections_path}")
bearer_token <- str_interp("Bearer ${access_token}")
res <- POST(
    url=url, body=toJSON(collection_form_metadata),
    add_headers(`Authorization`=bearer_token, `Content-Type`="application/json")
)
stop_for_status(res)
res_content <- content(res)
collection_id <- res_content$id
print("New private Collection id:")
print(collection_id)
print("New private Collection url:")
print(str_interp("${site_url}/collections/${collection_id}"))